## 선형 회귀분석

### 실습 데이터셋 준비

In [ ]:
# 관련 라이브러리를 호출합니다.
import os, joblib
import numpy as np
import pandas as pd
from scipy import stats

In [ ]:
# 시각화 관련 모듈을 호출합니다.
from GraphicSetting import *
import HelloDataScience as hds

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir('../data')

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 출력합니다.
os.listdir()

In [ ]:
# z 파일을 읽고 데이터프레임 df를 생성합니다.
df = joblib.load(filename = 'Used_Cars_Price_Prep.z')

In [ ]:
# df의 정보를 확인합니다.
df.info()

In [ ]:
# df의 처음 5행을 출력합니다.
df.head()

### 더미변수 생성

In [ ]:
# FuelType으로 더미변수를 생성합니다.
dm = pd.get_dummies(data = df['FuelType'], drop_first = True)

In [ ]:
# df의 오른쪽에 더미변수 dm을 추가합니다.
df = pd.concat(objs = [df, dm], axis = 1)

In [ ]:
# df의 처음 10행을 출력합니다.
df.head(n = 10)

In [ ]:
# df에서 FuelType을 삭제합니다.
df = df.drop(labels = ['FuelType'], axis = 1)

In [ ]:
# df의 정보를 확인합니다.
df.info()

In [ ]:
# MetColor 원소의 자료형을 정수로 변환합니다.
df['MetColor'] = df['MetColor'].astype('uint8')

In [ ]:
# df의 처음 10행을 출력합니다.
df.head(n = 10)

### 실습 데이터셋 분할

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn.model_selection import train_test_split

In [ ]:
# 목표변수 열이름을 설정합니다.
yvar = 'Price'

In [ ]:
# 입력변수 행렬 X와 목표변수 벡터 y를 생성합니다.
X = df.drop(labels = [yvar], axis = 1)
y = df[yvar]

In [ ]:
# 전체 데이터의 70%를 훈련셋, 30%를 시험셋으로 분할합니다.
Xtr, Xte, ytr, yte = train_test_split(
    X, y, test_size = 0.3, random_state = 0
)

In [ ]:
# 훈련셋의 목표변수 평균을 확인합니다.
ytr.mean()

In [ ]:
# 시험셋의 목표변수 평균을 확인합니다.
yte.mean()

### 선형 회귀모형 적합

In [ ]:
# 관련 라이브러리를 호출합니다.
import statsmodels.api as sm

In [ ]:
# 선형 회귀모형을 반환하는 함수를 정의합니다.
def ols(y, X):
    model = sm.OLS(endog = y, exog = X)
    return model.fit()

In [ ]:
# 훈련셋과 시험셋 입력변수에 y절편의 역할을 수행할 상수 1을 추가합니다.
Xtr = sm.add_constant(data = Xtr)
Xte = sm.add_constant(data = Xte)

In [ ]:
# 훈련셋 입력변수의 처음 5행을 출력합니다.
Xtr.head()

In [ ]:
# 훈련셋으로 선형 회귀모형을 적합합니다.
fit0 = ols(y = ytr, X = Xtr)

### 선형 회귀모형 적합 결과 확인

In [ ]:
# fit0 모형의 적합 결과를 확인합니다.
fit0.summary()

### 회귀진단: 잔차 가정 확인

In [ ]:
# fit0 모형 잔차의 등분산성 검정을 실행합니다.
hds.breushpagan(model = fit0)

### 회귀진단: 잔차 그래프

In [ ]:
# fit0 모형의 잔차 그래프를 그립니다.
hds.regressionDiagnosis(model = fit0)

### 회귀진단: 잔차의 정규성 검정

In [ ]:
# 잔차의 히스토그램을 그려서 분포를 확인합니다.
sns.histplot(x = fit0.resid, bins = 50, stat = 'density');

In [ ]:
# 잔차의 정규성 검정을 실행합니다.
stats.shapiro(x = fit0.resid)

### [참고] 영향점 확인

In [ ]:
# 훈련셋의 관측값마다 영향점 정보를 갖는 데이터프레임을 생성합니다.
aug = hds.augment(model = fit0)

In [ ]:
# aug의 정보를 확인합니다.
aug.info()

In [ ]:
# aug의 처음 5행을 출력합니다.
aug.head()

In [ ]:
# 스튜던트 잔차의 절대값이 3을 초과하는 행 개수를 확인합니다.
aug[np.abs(aug['std_resid']).gt(3)].shape[0]

In [ ]:
# 레버리지 평균을 계산합니다.
hatAvg = aug['hat'].mean()

In [ ]:
# 레버리지 평균의 3배를 초과하는 행 개수를 확인합니다.
aug[aug['hat'].gt(hatAvg * 3)].shape[0]

In [ ]:
# 훈련셋의 행 개수 n을 생성합니다.
n = Xtr.shape[0]

In [ ]:
# 쿡의 거리가 4/n(행 개수)를 초과하는 행 개수를 확인합니다.
aug[aug['cooksd'].gt(4/n)].shape[0]

### 훈련셋에서 이상치 제거

In [ ]:
# 쿡의 거리가 4/n(행 개수)를 초과하는 True, 아니면 False인 locs를 생성합니다.
locs = aug['cooksd'].gt(4/n)

In [ ]:
# 훈련셋에서 locs가 True인 이상치를 제거합니다.
Xtr1 = Xtr.loc[~locs]
ytr1 = ytr.loc[~locs]

### 선형 회귀모형 재적합 및 결과 확인

In [ ]:
# 이상치를 제거한 훈련셋으로 선형 회귀모형을 적합합니다.
fit1 = ols(y = ytr1, X = Xtr1)

In [ ]:
# fit1 모형의 적합 결과를 확인합니다.
fit1.summary()

In [ ]:
# fit1 모형 잔차의 등분산성 검정을 실행합니다.
hds.breushpagan(model = fit1)

In [ ]:
# fit1 모형의 잔차 그래프를 그립니다.
hds.regressionDiagnosis(model = fit1)

### [참고] 더미변수의 시각적 이해

In [ ]:
# Age와 Price의 회귀직선은 FuelType에 따라 달라집니다.
labels = ['Diesel', 'Petrol']
for i, v in enumerate(labels):
    sns.regplot(data = df[df['Petrol'].eq(i)], 
                x = 'Age', y = 'Price', 
                ci = None, label = v, 
                scatter_kws = dict(s = 10, alpha = 0.2))
plt.legend(loc = 'best', title = 'FuelType');

### 목표변수의 추정값 생성

In [ ]:
# 훈련셋의 실제값과 fit0 및 fit1 모형의 추정값을 비교합니다.
ytrPred0 = fit0.predict(exog = Xtr)
ytrPred1 = fit1.predict(exog = Xtr1)
pd.DataFrame(data = {'real': ytr, 'pred0': ytrPred0, 'pred1': ytrPred1})

In [ ]:
# 시험셋의 실제값과 fit0 및 fit1 모형의 추정값을 비교합니다.
ytePred0 = fit0.predict(exog = Xte)
ytePred1 = fit1.predict(exog = Xte)
pd.DataFrame(data = {'real': yte, 'pred0': ytePred0, 'pred1': ytePred1})

### 회귀모형 성능 평가

In [ ]:
# 훈련셋으로 fit0 모형의 성능지표를 출력합니다.
hds.regmetrics(y_true = ytr, y_pred = ytrPred0)

In [ ]:
# 훈련셋으로 fit1 모형의 성능지표를 출력합니다.
hds.regmetrics(y_true = ytr1, y_pred = ytrPred1)

In [ ]:
# 시험셋으로 fit0 모형의 성능지표를 출력합니다.
hds.regmetrics(y_true = yte, y_pred = ytePred0)

In [ ]:
# 시험셋으로 fit1 모형의 성능지표를 출력합니다.
hds.regmetrics(y_true = yte, y_pred = ytePred1)

### [참고] MSE, MAE, MAPE 관련 함수

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn import metrics

In [ ]:
# 시험셋의 MSE를 출력합니다.
metrics.mean_squared_error(y_true = yte, y_pred = ytePred1)

In [ ]:
# 시험셋의 MAE를 출력합니다.
metrics.mean_absolute_error(y_true = yte, y_pred = ytePred1)

In [ ]:
# 시험셋의 MAPE를 출력합니다.
metrics.mean_absolute_percentage_error(y_true = yte, y_pred = ytePred1)

### 다중공선성 확인

In [ ]:
# 분산팽창지수를 출력하고 다중공선성 입력변수를 확인합니다.
hds.vif(X = Xtr1)

In [ ]:
# 다중공선성 입력변수가 있다고 가정하고 훈련셋에서 삭제합니다.
Xtr2 = Xtr1.drop(labels = ['Petrol'], axis = 1)

In [ ]:
# 입력변수별 분산팽창지수를 다시 출력합니다.
hds.vif(X = Xtr2)

### 단계적방법으로 선형 회귀모형 적합

In [ ]:
# 단계적방법으로 선형 회귀모형을 적합합니다.
fit2 = hds.stepwise(y = ytr1, X = Xtr1, direction = 'both')

In [ ]:
# fit2 모형의 적합 결과를 확인합니다.
fit2.summary()

In [ ]:
# fit2 모형 잔차의 등분산성 검정을 실행합니다.
hds.breushpagan(model = fit2)

In [ ]:
# fit2 모형의 잔차 그래프를 그립니다.
hds.regressionDiagnosis(model = fit2)

### 표준화 회귀계수 확인

In [ ]:
# fit2 모형의 회귀계수를 출력합니다.
fit2.params

In [ ]:
# 표준화 회귀계수를 생성합니다.
beta_z = hds.std_coefs(model = fit2)

In [ ]:
# 표준화 회귀계수를 출력합니다.
beta_z

In [ ]:
# 표준화 회귀계수의 절대값을 오름차순 정렬한 결과를 출력합니다.
beta_z.abs().sort_values()

### 회귀모형 성능 평가

In [ ]:
# 시험셋으로 fit2 모형의 추정값을 생성합니다.
ytePred2 = fit2.predict(exog = Xte)

In [ ]:
# 시험셋으로 fit2 모형의 성능지표를 출력합니다.
hds.regmetrics(y_true = yte, y_pred = ytePred2)

In [ ]:
# fit1 모형의 성능지표와 비교합니다.
hds.regmetrics(y_true = yte, y_pred = ytePred1)

## End of Document